In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class mLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size, output_size):
        super(mLSTM, self).__init__()

        self.hidden_size = hidden_size
        # input embedding
        self.encoder = nn.Embedding(input_size, embed_size)
        # lstm weights
        self.weight_fm = nn.Linear(hidden_size, hidden_size)
        self.weight_im = nn.Linear(hidden_size, hidden_size)
        self.weight_cm = nn.Linear(hidden_size, hidden_size)
        self.weight_om = nn.Linear(hidden_size, hidden_size)
        self.weight_fx = nn.Linear(embed_size, hidden_size)
        self.weight_ix = nn.Linear(embed_size, hidden_size)
        self.weight_cx = nn.Linear(embed_size, hidden_size)
        self.weight_ox = nn.Linear(embed_size, hidden_size)
        # multiplicative weights
        self.weight_mh = nn.Linear(hidden_size, hidden_size)
        self.weight_mx = nn.Linear(embed_size, hidden_size)
        # decoder
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, inp, h_0, c_0):
        # encode the input characters
        inp = self.encoder(inp)
        # calculate the multiplicative matrix
        m_t = self.weight_mx(inp) * self.weight_mh(h_0)
        # forget, input and output gates
        f_g = torch.sigmoid(self.weight_fx(inp) + self.weight_fm(m_t))
        i_g = torch.sigmoid(self.weight_ix(inp) + self.weight_im(m_t))
        o_g = torch.sigmoid(self.weight_ox(inp) + self.weight_om(m_t))
        # intermediate cell state
        c_tilda = torch.tanh(self.weight_cx(inp) + self.weight_cm(m_t))
        # current cell state
        cx = f_g * c_0 + i_g * c_tilda
        # hidden state
        hx = o_g * torch.tanh(cx)

        out = self.decoder(hx.view(1,-1))

        return out, hx, cx

    def init_hidden(self):
        h_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        c_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        return h_0, c_0



In [2]:
# Converte string para uma lista de inteiros
from unidecode import unidecode
import random

def generate_hidden(text, rnn):
    hidden, cell = rnn.init_hidden()
    text_tensor = char_tensor(text).cuda()
    for p in range(len(text)):
        output, hidden, cell = rnn(text_tensor[p], hidden, cell)
    return hidden, cell

def char_tensor(string):
    string = unidecode(string)
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = ord(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('The omega (Ω) symbol\n'))  

tensor([ 84, 104, 101,  32, 111, 109, 101, 103,  97,  32,  40,  79,  41,  32,
        115, 121, 109,  98, 111, 108,  10])


In [3]:
embed_size = 128 # ascii representation
hidden_size = 2048
rnn = mLSTM(embed_size, hidden_size, embed_size, embed_size).cuda()
rnn.load_state_dict(torch.load("lstm_11.pth"))
rnn.eval()

hidden, cell = generate_hidden("Adorei! Chegou bem cedo!", rnn)


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = mLSTM(embed_size, hidden_size, embed_size, embed_size).cuda()
        self.rnn.load_state_dict(torch.load("lstm_11.pth"))
        self.fc1 = nn.Linear(2*hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, int(hidden_size/4))
        self.fc3 = nn.Linear(int(hidden_size/4), int(hidden_size/2))
        self.fc4 = nn.Linear(int(hidden_size/2), 5)
        self.dropout = nn.Dropout(0.5) 
        
    def forward(self, text_tensor):
        with torch.no_grad():
            hidden, cell = rnn.init_hidden()
            for p in range(len(text_tensor)):
                _, hidden, cell = rnn(text_tensor[p], hidden, cell)
        
        
        x = torch.cat((hidden, cell), 0).view(1, -1)
        x = F.relu(self.dropout(self.fc1(x)))
        x = F.relu(self.dropout(self.fc2(x)))
        x = F.relu(self.dropout(self.fc3(x)))
        x = F.relu(self.fc4(x))
        return x

In [5]:
import pandas as pd
df_test = pd.read_csv("../data/interim/test.csv")

In [6]:
test_data = dict()
for score, message in zip(df_test.score.values, df_test.message.values):
    try:
        test_data[score].append([score, message])
    except:
        test_data[score] = [[score, message]]

In [7]:
net = Net()
net.load_state_dict(torch.load("net_9.pth"))
net = net.cuda()
net.eval()

Net(
  (rnn): mLSTM(
    (encoder): Embedding(128, 128)
    (weight_fm): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_im): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_cm): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_om): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_fx): Linear(in_features=128, out_features=2048, bias=True)
    (weight_ix): Linear(in_features=128, out_features=2048, bias=True)
    (weight_cx): Linear(in_features=128, out_features=2048, bias=True)
    (weight_ox): Linear(in_features=128, out_features=2048, bias=True)
    (weight_mh): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_mx): Linear(in_features=128, out_features=2048, bias=True)
    (decoder): Linear(in_features=2048, out_features=128, bias=True)
  )
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
  (fc3): Linear(in_features=5

In [8]:
from tqdm import tqdm
pred, y, messages = list(), list(), list()
for score in test_data:
    for _, message in tqdm(test_data[score], total=len(test_data[score])):
        y.append(score)
        messages.append(message)
        pred.append(net(char_tensor(message).cuda()))

100%|██████████| 698/698 [01:02<00:00,  9.80it/s]


In [9]:
def get_score(prediction):
    hiscore, higrade = 1, 0
    for score, e in enumerate(list(prediction.detach()[0])):
        if e.item() > higrade:
            hiscore = score+1
            higrade = e.item()
    return hiscore#, higrade

In [10]:
pred = list(map(get_score, pred))

In [23]:
correct5 = incorrect5 =  correct1 = incorrect1 = 0
for p, y_value, message in zip(pred, y, messages):
    if p == y_value:
        if p == 1:
            correct1 += 1
        else:
            correct5 += 1
    else:
        if y_value == 1:
            incorrect1 += 1
        elif y_value == 5:
            incorrect5 += 1
        print(p, y_value, message)

1 5 Preciso devolver o produto pois essa compra havia sido cancelada. Acredito que tenha sido problema do sistema. Estou aguardando resposta para devolver e pegar o dinheiro de volta
1 5 Além de muita organização desde a emissão da nota fiscal o produto foi entregue sem nenhuma avaria!
1 5 Parece que vamos ter que trocar o produto. A boia veio furada.
1 5 Recebi o produto bem antes do prazo. Ainda não foi testado, pois, é presente e não foi entregue.
1 5 Comprei o produto com garantia estendida de um ano e o produto chegou totalmente quebrado, quero saber o procedimento de devolução o mais rápido possível 
1 5 Recebi a mercadoria antes do prazo e em condições muito boa.

Só não instalei o testar ainda o produto!
1 5 Amei minha compra. lindaa a capa que peguei 😍😍😍 e resistente. Recomendo ?????
1 5 Chegou correto. Vou instalar para testar agora!
1 5 Recebi num prazo menor que o estimado, produto conforme anunciado, com nota fiscal.
1 5 RECEBI DIREITINHO E RAPIDO. COMPRAREI NOVAMENTE.
1 5

5 4 Como na foto 
1 4 Baixar o valor do frete, tem sido um pouco caro
5 4 Sempre muito bem atendida!
5 4 Muito satisfeito com a compra e o atendimento
1 4 Hidrata bem porém não alisa muito 
5 4 Produto bom e luja confiável chegou antes do previsto tudo certinho loja Boa 
5 4 A ENTREGA FOI SUPER RÁPIDA AMEI 
5 4 confiável
5 4 Antes do prazo.
5 4 Eu recomendo ! No momento não tenho nada contra .
5 4 Amei minha cortina é linda e chegou bem antes do prazo
5 4 Recebi exatamente o que esperava. As demais encomendas de outros vendedores atrasaram, mas esta chegou no prazo.
5 4 Entrega antes do prazo previsto.
1 4 Eu adquiri 2 (duas) unidades do produto, mas só enviaram (1) uma, porém na NFe enviada na caixa diz que foi faturado 2 (dois) produtos e foi debitado no meu cartão o valor total compra. Como resolvo?
5 4 Havíamos encomendado 2 unidades deste produto apenas chegou uma ...mas após 5 dias chegou a outro ..e tudo certo ...e o produto é maravilhoso ,recomendo .
5 4 foi muito bom comprar c

5 3 bom produto achei um pouco fina a toalha. Falta na hora da compra a marca do fabricante


5 3 Tudo dentro do programado...entrega rapida...recebimento perfeito, só não posso avaliar o produto, pois ainda instalei.
5 3 Bom o meu produto que no caso era uma fonte para computador não foi entregue. porem eles tem um otimo atendimento.
5 3 Esperava a impressão com mais qualidade mas no geral o produto é muito bom.
5 3 Ótima compra chegou bem antes do prazo, ainda não usei mas fiquei hiper feliz recomendo 👏👏👏👏👏
1 3 Na hora da compra após colocar cartão de crédito e confirmar vai para uma tela escrita em inglês que parece erro. Segunda vez que ocorre. Da outra vez pensei que não tinha concluído compra e a fiz .
5 3 sempre compro neste site muito bom
1 3 A caixa veio amassada e molhada. Veio quatro frasco de 500 ML e um veio trincado e vazou metade do produto.
1 3 Só uma coisa que me confundiu eles entregam primeiro a nota e depois o produto. Aí achei que a encomenda tinha sido entregue er

In [24]:
correct5, incorrect5, correct1, incorrect1

(3265, 260, 1629, 141)

In [12]:
correct, incorrect

(4894, 2617)

1 4 Mosquitos frustrados pela boa qualidade do Mosquitero. Nunca mais acordar cantando.
1 4 A base é pequena. Não sei se um bauleto ficará firme.
5 4 Recomendo a todos!
5 4 Tudo dentro do previsto 
5 4 Bons panos de pratos !
5 4 A entrega até foi antes do previsto..gostei muito
1 4 Só posso avaliar quando receber o produto. Ainda está no prazo de entrega informado. 
1 4 Solicito a evitarem o envio de tanta papelada de propaganda junto, em muito repetitivos a cada compra. 
Lembrem-se: NÃO HÁ COMO SE TER PAZ SEM ECOLOGIA.

In [13]:
bias = dict()
for p in pred:
    try:
        bias[p]+=1
    except:
        bias[p]=1

In [14]:
bias

{5: 4490, 1: 3021}

In [26]:
correct5 = incorrect5 =  correct1 = incorrect1 = 0
for p, y_value, message in zip(pred, y, messages):
    if y_value in {1,2,3}:
        if p == 1:
            correct1 += 1
        else:
            incorrect1 += 1
    else:
        if p == 5:
            correct5 += 1
        else:
            incorrect5 += 1
        

In [27]:
correct5, incorrect5, correct1, incorrect1

(4027, 579, 2442, 463)

In [28]:
4027 / (4027+579)

0.8742943986105081

In [29]:
2442 / (2442 + 463)

0.8406196213425129

In [30]:
correct5 = incorrect5 =  correct1 = incorrect1 = 0
for p, y_value, message in zip(pred, y, messages):
    if y_value in {1,2}:
        if p == 1:
            correct1 += 1
        else:
            incorrect1 += 1
    else:
        if p == 5:
            correct5 += 1
        else:
            incorrect5 += 1
        